In [100]:
import numpy as np
import pandas as pd 

pd.options.display.max_columns = None
pd.options.display.width = None
pd.set_option('display.max_rows', 50)

In [101]:
!kaggle datasets download -d sudalairajkumar/cryptocurrency-historical-prices-coingecko -f bitcoin.csv --force -p data
!kaggle datasets download -d sudalairajkumar/cryptocurrency-historical-prices-coingecko -f ethereum.csv --force -p data

100%|█████████████████████████████████████████| 277k/277k [00:00<00:00, 645kB/s]
100%|█████████████████████████████████████████| 277k/277k [00:00<00:00, 643kB/s]
100%|█████████████████████████████████████████| 268k/268k [00:00<00:00, 635kB/s]
100%|█████████████████████████████████████████| 268k/268k [00:00<00:00, 634kB/s]


In [102]:
shares = ["bitcoin", "ethereum"]
tickers_str = ', '.join(shares)
all_dfs = pd.DataFrame()

for ticker in shares:
    df = pd.read_csv(f"data/{ticker}.csv", index_col=["date"])
    df = df.add_suffix(f'_{ticker}')
    all_dfs = pd.concat([all_dfs, df[f'price_{ticker}']], axis=1).dropna()
all_dfs.sort_index(axis=0)

,price_bitcoin,price_ethereum
2015-08-07 00:00:00.000,278.508814,2.831620
2015-08-08 00:00:00.000,259.800728,1.330750
2015-08-10 00:00:00.000,263.578200,0.687586
2015-08-11 00:00:00.000,269.866800,1.067379
2015-08-12 00:00:00.000,267.712600,1.256613
...,...,...
2024-02-09 00:00:00.000,45338.271190,2420.968561
2024-02-10 00:00:00.000,47143.242991,2487.850897
2024-02-11 00:00:00.000,47768.995251,2499.492184
2024-02-12 00:00:00.000,48189.881793,2505.076808


In [103]:
btc = x = np.array(list(all_dfs['price_bitcoin'])).reshape((-1, 1))  # независимая переменная, предсказание
eth = y = np.array(list(all_dfs['price_ethereum']))  # зависимая переменная, ответ

In [104]:
100*all_dfs['price_bitcoin'].corr(all_dfs['price_ethereum'])

94.17748406839648

In [105]:
from sklearn.linear_model import LinearRegression

In [106]:
model = LinearRegression().fit(x, y)

In [107]:
print('coefficient of determination R^2:', model.score(x, y))
print('intercept:', model.intercept_)
print('coef:', model.coef_[0])

coefficient of determination R^2: 0.8869398505453068
intercept: -85.72707862678988
coef: 0.06250335530270418


In [108]:
x_new = np.array([43000, 44000, 45000, 50000]).reshape((-1, 1))
y_new = model.predict(x_new).round(2)

In [109]:
print(f'predicted response:', *y_new, sep='\n')

predicted response:
2601.92
2664.42
2726.92
3039.44


In [110]:
all_dfs['price_ethereum_clear'] = model.intercept_ + model.coef_[0] * all_dfs['price_bitcoin']
all_dfs

,price_bitcoin,price_ethereum,price_ethereum_clear
2015-08-07 00:00:00.000,278.508814,2.831620,-68.319343
2015-08-08 00:00:00.000,259.800728,1.330750,-69.488661
2015-08-10 00:00:00.000,263.578200,0.687586,-69.252557
2015-08-11 00:00:00.000,269.866800,1.067379,-68.859498
2015-08-12 00:00:00.000,267.712600,1.256613,-68.994143
...,...,...,...
2024-02-09 00:00:00.000,45338.271190,2420.968561,2748.066994
2024-02-10 00:00:00.000,47143.242991,2487.850897,2860.883788
2024-02-11 00:00:00.000,47768.995251,2499.492184,2899.995404
2024-02-12 00:00:00.000,48189.881793,2505.076808,2926.302225
